### Imports
Event accumulator to fetch data from a saved checkpoint file

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
event_file = '../train/log_000'
eacc = EventAccumulator(event_file)
eacc.Reload()
eacc.Tags()

In [ ]:
np.set_printoptions(precision=3)

model_name = 'XOR_model'
layer_name = 'output_layer'
data_type = np.float32
for tt in eacc.Tags()['tensors']:
    fetch = '/'.join([model_name, layer_name])
    if fetch in tt:
        print(tt)
        for te in eacc.Tensors(tt):
            shape = [i.size for i in te.tensor_proto.tensor_shape.dim]
            binary_tensor = te.tensor_proto.tensor_content
            deci_tensor  = np.fromstring(binary_tensor, dtype=np.float32).reshape(shape)
            print('training step: {}\n tensor shape: {}\n{}\n'.format(te.step, shape, deci_tensor))
    if 'input_patterns' in tt or 'target_patterns' in tt:
        print(tt)
        for te in eacc.Tensors(tt):
            shape = [i.size for i in te.tensor_proto.tensor_shape.dim]
            binary_tensor = te.tensor_proto.tensor_content
            deci_tensor  = np.fromstring(binary_tensor, dtype=np.float32).reshape(shape)
            print('training step: {}\n tensor shape: {}\n{}\n'.format(te.step, shape, deci_tensor))
    if 'input_patterns' in tt or 'target_patterns' in tt:
        print(tt)
        for te in eacc.Tensors(tt):
            print(te)
#             shape = [i.size for i in te.tensor_proto.tensor_shape.dim]
#             binary_tensor = te.tensor_proto.tensor_content
#             deci_tensor  = np.fromstring(binary_tensor, dtype=np.float32).reshape(shape)
#             print('training step: {}\n tensor shape: {}\n{}\n'.format(te.step, shape, deci_tensor))


In [ ]:
def get_event_accumulator(path_to_event_file):
    event_file = path_to_event_file
    eacc = EventAccumulator(event_file)
    eacc.Reload()
    return eacc

def retrieve_array(path_to_event_file, layer_name, tensor_name, data_type=np.float32):
    eacc = get_event_accumulator(path_to_event_file)
    ln = layer_name
    dt = data_type
    for tt in eacc.Tags()['tensors']:
        fetch = '/'.join([layer_name, tensor_name])
        if fetch in tt:
            ra = []
            rs = []
            for te in eacc.Tensors(tt):
                shape = [i.size for i in te.tensor_proto.tensor_shape.dim]
                binary_tensor = te.tensor_proto.tensor_content
                deci_tensor  = np.fromstring(binary_tensor, dtype=dt).reshape(shape)
                ra.append(deci_tensor)
                rs.append(te.step)
    return ra, rs

In [ ]:
retrieve_array(event_file, 'hidden_layer', 'biases')

In [ ]:
retrieve_array(event_file, 'hidden_layer', 'net_input')

In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib notebook

In [25]:
logdir = '/Users/alexten/Projects/pdpyflow/xor/train/log_000'
event_file = os.path.join(logdir, os.listdir(logdir)[0])
for i, event in enumerate(tf.train.summary_iterator(event_file)):
    print(i, type(event))

0 <class 'tensorflow.core.util.event_pb2.Event'>
1 <class 'tensorflow.core.util.event_pb2.Event'>
2 <class 'tensorflow.core.util.event_pb2.Event'>
3 <class 'tensorflow.core.util.event_pb2.Event'>
4 <class 'tensorflow.core.util.event_pb2.Event'>
5 <class 'tensorflow.core.util.event_pb2.Event'>
6 <class 'tensorflow.core.util.event_pb2.Event'>
7 <class 'tensorflow.core.util.event_pb2.Event'>
8 <class 'tensorflow.core.util.event_pb2.Event'>
9 <class 'tensorflow.core.util.event_pb2.Event'>
10 <class 'tensorflow.core.util.event_pb2.Event'>
11 <class 'tensorflow.core.util.event_pb2.Event'>
12 <class 'tensorflow.core.util.event_pb2.Event'>
13 <class 'tensorflow.core.util.event_pb2.Event'>


DataLossError: truncated record at 122274